In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

/home/chenxiaoyun/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/chenxiaoyun/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

In [3]:
tf.__version__

'1.8.0'

### 定义神经网络所需要的基本参数

In [4]:
# 第一层卷积层的卷积核大小
filter_size1 = 5
# 第一层卷积核的个数
num_filters1 = 16
# 第二层卷积核的大小
filter_size2 = 5
# 第二层卷积核的大小
num_filter2 = 36
# 全连接层的大小
fc_size = 128

### 载入MNIST 数据集

In [5]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

In [6]:
# 查看数据集的大小
print("训练集的大小：", data.num_train)
print("验证集的大小:", data.num_val)
print("测试集的大小:", data.num_test)

训练集的大小： 55000
验证集的大小: 5000
测试集的大小: 10000


### 复制数据维度信息

In [7]:
# 图片展开的大小
img_size_flat = data.img_size_flat

#图片原始尺寸
img_shape = data.img_shape

# 类别数
num_classes = data.num_classes

### Tensorflow图简单说明
- 占位符变量(placeholder)用来改变图的输入
- 模型变量(Model)将会被优化，使得模型的表现更好
- 模型本质上就是一些数学函数，它根据placeholder和模型的输入变量来计算一些输出
- 一个cost度量用来指导变量的优化
- 一个优化策略会更新模型的变量

### 创建新变量的帮助函数

In [9]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [15]:
tf.truncated_normal??

In [12]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [14]:
tf.constant??

### 创建卷积层的帮助函数

假设输入的四维张量，各个维度如下

1.图像数量 

2.每张图像的Y轴 

3.每张图像的X轴 

4.每张图像的通道数 


输入图片的张量的时候可能是彩色的通道即红绿蓝，当输入是前一层卷积层生成的输出的时候，它可能是滤波通道

输出是另外一个四通道的张量，如下
1.图像数量，与输入相同 

2.每张图的Y轴，如果用了2x2的池化，则输入是图像宽高的一半 

3.每张图像的X轴 

4.卷积滤波生成的通道数 


In [ ]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    # 定义输入张量的形状
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    # 初始化当前层的weights
    weights = new_weights(shape=shape)
    # 初始化biases
    biases = new_biases(length=num_filters)
    # 建立一个卷积
    layer = tf.layers.conv2d(
        input=input,
        filter=weights,
        strides=[1, 1, 1, 1],
        padding='SAME'
    )
    layer += biases
    if use_pooling is True:
        # 进行池化层操作
        # 初始化参数为： 输入层、卷积核大小2x2，步长高宽2，2，填充0
        layer = tf.nn.max_pool(
            value=layer,
            ksize=[1, 2, 2, 1],
            strides=[1, 2, 2, 1],
            padding='SAME'
        )
    # 对卷积的输出或池化层进行激活
    tf.nn.relu(layer)
    